## Task 1 - Virtual Environment

#### 1.1 Change to a Free GPU Runtime

In [1]:
# TASK 1.1: Check if you are currently using the GPU in Colab

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

#### TASK 1.2: Mount Google Drive

In [0]:
# Task: 1.2.1 Install google-drive-ocamlfuse

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [3]:
# Task: 1.2.2 Authenticate and get credentials

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
# TASK 1.2.3: Mount Google Drive in local Colab VM

!mkdir -p drive
!google-drive-ocamlfuse drive
!ls

adc.json  drive  sample_data


In [5]:
!ls drive

Apps				      print
Colab Notebooks			      Telebot +.ods
NTUOSS-ImageRecognitionWorkshop       TGIFHacks #82 - Dogs vs Cats.odt
NTUOSS-ImageRecognitionWorkshop-Data  Tomato & Egg


In [6]:
!ls drive/NTUOSS-ImageRecognitionWorkshop

start.ipynb


In [12]:
!ls drive/NTUOSS-ImageRecognitionWorkshop-Data

model  test  train  validation


## Task 2 - Preprocess Images

#### 2.1 Configure Image Augmentation

In [9]:
# TASK 2.1 : Add augmentation configuration for the data generator of train data only

from keras.preprocessing.image import ImageDataGenerator

datagen_train =  ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = 30,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)
datagen_val = ImageDataGenerator(
    rescale = 1. / 255)

Using TensorFlow backend.


#### 2.2 Generate Image Data from Directory

In [13]:
# TASK 2.2.1 : Generate Image Data from Directory and Set parameter

train_data = datagen_train.flow_from_directory(
    directory = './drive/NTUOSS-ImageRecognitionWorkshop-Data/train',
    target_size = (150, 150),
    class_mode = 'binary',
    shuffle = True,
    batch_size = 50)
validation_data = datagen_val.flow_from_directory(
    directory = './drive/NTUOSS-ImageRecognitionWorkshop-Data/validation',
    target_size = (150, 150),
    class_mode = 'binary',
    shuffle = True,
    batch_size = 50)

Found 4000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [14]:
# TASK 2.2.2 : Check class indices
print('Class Indices : {}'.format(train_data.class_indices))

Class Indices : {'cat': 0, 'dog': 1}


## Task 3 - Build Basic Model

#### 3.1 Set up backend and Import libraries

In [0]:
# TASK 3.1.1 Configure backend

from keras import backend as K
K.set_image_dim_ordering('tf') #channel last
K.set_image_data_format('channels_last')

In [0]:
# TASK 3.1.2 Import the Keras libraries and packages

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

#### 3.2 Construct Model

In [0]:
# TASK 3.2.1 Initialize Neural Network Model
model = Sequential()

# TASK 3.2.2 Create first set of CONV -> RELU -> POOL layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# TASK 3.2.3 Create second set of CONV -> RELU -> POOL layers
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# TASK 3.2.4 Create third set of CONV -> RELU -> POOL layers
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))

# TASK 3.2.5 Convert the 3D feature maps to 1D feature vectors
model.add(Flatten())

# TASK 3.2.6 Add the connection layer
model.add(Dense(units = 256, activation='relu'))

# TASK 3.2.7 Add the output layer
model.add(Dense(units = 1, activation = 'sigmoid'))

# TASK 3.2.8 Compile the model
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

#### 3.3 Check Model

In [0]:
# TASK 3.3 Check the model structure.
model.summary()

## Task 4 - Train Model



In [0]:
# TASK 4: Train Model [WARNING: It took me 84.95 min]
import time
train_start_time = time.time()

from keras.callbacks import EarlyStopping
model.fit_generator(
    generator = train_data,
    epochs = 25,
    callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)],
    validation_data = validation_data)
model.save('./drive/NTUOSS-ImageRecognitionWorkshop/cnn_model_basic.h5') # save model
print("It takes {:.2f} min to train the model".format((time.time() - train_start_time)/60 ))

## Task 5 - Test Basic Model

#### 5.1 Load Model

In [0]:
# Skip this if you just finished training the model and the model exists in the working environment

# TASK 5.1: Load trained model
from keras.models import load_model
model_basic = load_model('./drive/NTUOSS-ImageRecognitionWorkshop-Data/model/cnn_model_basic.h5')
# model_basic = load_model('./drive/NTUOSS-ImageRecognitionWorkshop/cnn_model_basic.h5') # To load the model you train

#### 5.2 Generate Test Image Data from Directory

In [23]:
# TASK 5.2.1: Set up data generator for test data
from keras.preprocessing.image import ImageDataGenerator
datagen_test = ImageDataGenerator(rescale=1. / 255)

test_data = datagen_test.flow_from_directory (
    directory = './drive/NTUOSS-ImageRecognitionWorkshop-Data/test',
    target_size = (150, 150),
    class_mode = None,
    shuffle = False,
    batch_size = 200)


Found 200 images belonging to 2 classes.


In [24]:
# TASK 5.2.2: Check test generator
print(test_data.class_indices)
print(test_data.filenames)

{'cat': 0, 'dog': 1}
['cat/cat.1029.jpg', 'cat/cat.10603.jpg', 'cat/cat.10629.jpg', 'cat/cat.10659.jpg', 'cat/cat.10731.jpg', 'cat/cat.1080.jpg', 'cat/cat.10805.jpg', 'cat/cat.10814.jpg', 'cat/cat.10824.jpg', 'cat/cat.10842.jpg', 'cat/cat.10844.jpg', 'cat/cat.10864.jpg', 'cat/cat.10878.jpg', 'cat/cat.10908.jpg', 'cat/cat.10913.jpg', 'cat/cat.10946.jpg', 'cat/cat.10952.jpg', 'cat/cat.10954.jpg', 'cat/cat.1096.jpg', 'cat/cat.11091.jpg', 'cat/cat.11097.jpg', 'cat/cat.11113.jpg', 'cat/cat.11137.jpg', 'cat/cat.11145.jpg', 'cat/cat.11158.jpg', 'cat/cat.11215.jpg', 'cat/cat.1123.jpg', 'cat/cat.11283.jpg', 'cat/cat.11301.jpg', 'cat/cat.11310.jpg', 'cat/cat.11370.jpg', 'cat/cat.11372.jpg', 'cat/cat.11399.jpg', 'cat/cat.11449.jpg', 'cat/cat.11474.jpg', 'cat/cat.11493.jpg', 'cat/cat.1150.jpg', 'cat/cat.11528.jpg', 'cat/cat.1157.jpg', 'cat/cat.1166.jpg', 'cat/cat.1180.jpg', 'cat/cat.1184.jpg', 'cat/cat.1233.jpg', 'cat/cat.1238.jpg', 'cat/cat.1261.jpg', 'cat/cat.1270.jpg', 'cat/cat.1299.jpg', 'cat/

#### 5.3 Make Prediction

In [0]:
# TASK 5.3.1: Use model to yield score prediction for test data
scores = model_basic.predict_generator(test_data)
print(scores)

In [27]:
# TASK 5.3.2: Process scores to get prediction result
y_pred = [round(score[0]) for score in scores]
print(y_pred)

[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]

In [28]:
# TASK 5.3.3: Prepare actual result using filenames
y_true = [0 if 'cat' in filename else 1 for filename in test_data.filenames]
print(y_true)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [29]:
# TASK 5.3.4: Calculate accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

0.77


In [30]:
# TASK 5.3.5: Generate a report
import pandas as pd
pd.crosstab(pd.Series(y_true), pd.Series(y_pred), rownames = ['True'], colnames = ['Pred'], margins = True)

Pred,0.0,1.0,All
True,,,
0,65,35,100
1,11,89,100
All,76,124,200


## Task 6 - Recognize Image with Basic Model

#### 6.1 Useful Functions

In [0]:
# TASK 6.1.1: Define a function for reading image from url
import requests, io
from PIL import Image
def read_image_from_url(url):
    try:
        r = requests.get(url, timeout=15)
        img = Image.open(io.BytesIO(r.content))
        return img
    except:
        print("{:<10} Cannot find image from {}".format('[ERROR]', url))
        exit(1)

In [0]:
# TASK 6.1.2: Define a function for preprocessing image
from PIL import Image
import numpy as np
def preprocess_image(img, target_size):
    from keras.preprocessing.image import img_to_array
    img = img.resize(target_size,Image.ANTIALIAS)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img.astype('float32')
    img /= 255
    return img

In [0]:
# TASK 6.1.3: Define a function for processing result
def process_result(score):
    return 'dog' if score > 0.5 else 'cat'

#### 6.2 Make Prediction

In [37]:
# TASK 6.2: Read image, Preprocess image, and Make prediction
image = read_image_from_url('https://www.readersdigest.ca/wp-content/uploads/2011/01/4-ways-cheer-up-depressed-cat.jpg') # replace with any image url
image = preprocess_image(image, (150, 150))
score = model_basic.predict(image)
print('Score: ' + str(score))
print('Class: ' + process_result(score))

Probability: [[0.3044719]]
Class: cat


## Task 7 - Build on Top of Pretrained Model

#### 7.1 Set up backend and Import libraries

In [0]:
# TASK 7.1.1 Configure backend
from keras import backend as K
K.set_image_dim_ordering('tf') #channel last
K.set_image_data_format('channels_last')

In [0]:
# TASK 7.1.2 Import the Keras libraries and packages
from keras.applications import VGG16
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model

#### 7.2 Construct Model

In [40]:
# TASK 7.2.1: Load base model
base_model = VGG16(include_top = False, weights = 'imagenet', input_shape = (150, 150, 3))

58892288/58889256 [==============================] - 4s 0us/step


In [0]:
# TASK 7.2.2: Add new layers
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x) #new FC layer, random init
x = Dropout(0.4)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [43]:
# TASK 7.2.3: Setup trainable layer
for layer in base_model.layers:
    layer.trainable = False
print("{:<10} Pretrained model layers: {}".format('[INFO]', len(base_model.layers)))
print("{:<10} Total number of layers : {}".format('[INFO]', len(model.layers)))


[INFO]     Pretrained model layers: 19
[INFO]     Total number of layers : 23


In [0]:
# TASK 7.2.4: Compile model
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

#### 7.3 Check Model

In [45]:
# TASK 7.3: Check Model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

## Task 8 - Train Advanced Model

In [0]:
# TASK 8: Train advanced model.

import time
train_start_time = time.time()

from keras.callbacks import EarlyStopping

model.fit_generator(
    train_data,
    epochs= 25 ,
    callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)],
    validation_data = validation_data)
model.save('./drive/NTUOSS-ImageRecognitionWorkshop/cnn_model_advanced.h5')

print("It takes {:.2f} min to train the model".format((time.time() - train_start_time)/60 ))

## Task 9 - Test Advanced Model

#### 9.1 Load Model

In [0]:
# Skip this if you just finished training the model and the model exists in the working environment

# TASK 5.1: Load trained model
from keras.models import load_model
model_advanced = load_model('./drive/NTUOSS-ImageRecognitionWorkshop-Data/model/cnn_model_advanced.h5')
# model_advanced = load_model('./drive/NTUOSS-ImageRecognitionWorkshop/cnn_model_advanced.h5') # To load the model you train

#### 9.2 Generate Test Image Data from Directory

In [48]:
# TASK 9.2.1 : Set up data generator for test data
from keras.preprocessing.image import ImageDataGenerator
datagen_test = ImageDataGenerator(rescale=1. / 255)

test_data = datagen_test.flow_from_directory (
    directory = './drive/NTUOSS-ImageRecognitionWorkshop-Data/test',
    target_size = (150, 150),
    class_mode = None,
    shuffle = False,
    batch_size = 200)  

Found 200 images belonging to 2 classes.


In [49]:
# TASK 9.2.2: Check test data generator
print(test_data.class_indices)
print(test_data.filenames)

{'cat': 0, 'dog': 1}
['cat/cat.1029.jpg', 'cat/cat.10603.jpg', 'cat/cat.10629.jpg', 'cat/cat.10659.jpg', 'cat/cat.10731.jpg', 'cat/cat.1080.jpg', 'cat/cat.10805.jpg', 'cat/cat.10814.jpg', 'cat/cat.10824.jpg', 'cat/cat.10842.jpg', 'cat/cat.10844.jpg', 'cat/cat.10864.jpg', 'cat/cat.10878.jpg', 'cat/cat.10908.jpg', 'cat/cat.10913.jpg', 'cat/cat.10946.jpg', 'cat/cat.10952.jpg', 'cat/cat.10954.jpg', 'cat/cat.1096.jpg', 'cat/cat.11091.jpg', 'cat/cat.11097.jpg', 'cat/cat.11113.jpg', 'cat/cat.11137.jpg', 'cat/cat.11145.jpg', 'cat/cat.11158.jpg', 'cat/cat.11215.jpg', 'cat/cat.1123.jpg', 'cat/cat.11283.jpg', 'cat/cat.11301.jpg', 'cat/cat.11310.jpg', 'cat/cat.11370.jpg', 'cat/cat.11372.jpg', 'cat/cat.11399.jpg', 'cat/cat.11449.jpg', 'cat/cat.11474.jpg', 'cat/cat.11493.jpg', 'cat/cat.1150.jpg', 'cat/cat.11528.jpg', 'cat/cat.1157.jpg', 'cat/cat.1166.jpg', 'cat/cat.1180.jpg', 'cat/cat.1184.jpg', 'cat/cat.1233.jpg', 'cat/cat.1238.jpg', 'cat/cat.1261.jpg', 'cat/cat.1270.jpg', 'cat/cat.1299.jpg', 'cat/

#### 9.3 Make Prediction

In [50]:
# TASK 9.3: Make prediction and Check model performance

# Use model to yield score prediction for test data
scores = model_advanced.predict_generator(test_data)

# Process scores to get prediction result
y_pred = [round(score[0]) for score in scores]

# Prepare actual result using filenames
y_true = [0 if 'cat' in filename else 1 for filename in test_data.filenames]

# Calculate accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

# Generate a report
import pandas as pd
pd.crosstab(pd.Series(y_true), pd.Series(y_pred), rownames = ['True'], colnames = ['Pred'], margins = True)


0.83


Pred,0.0,1.0,All
True,,,
0,70,30,100
1,4,96,100
All,74,126,200


## Task 10 - Recognize Image with Advanced Model

#### 10.1 Make Prediction

In [51]:
# TASK 10.1: Read image, Preprocess image, and Make prediction
image = read_image_from_url('https://www.readersdigest.ca/wp-content/uploads/2011/01/4-ways-cheer-up-depressed-cat.jpg') # replace with any image url
image = preprocess_image(image, (150, 150))
score = model_advanced.predict(image)
print('Score: ' + str(score))
print('Class: ' + process_result(score))

Score: [[0.18195733]]
Class: cat


In [0]:
# Restart Google Colab - Only run the line below when u mess up with the virtual machine
# !kill -9 -1